In [1]:
import pickle
import plotly.io as pio
import plotly.graph_objects as go
import plotly.express as px
import json
import pandas as pd
import numpy as np
import os
import glob
import os.path
import plotly.graph_objs as go
from itertools import product
from plotly.subplots import make_subplots
import  plotly.graph_objs as go 
import plotly
from itertools import product
from statsmodels.sandbox.stats.multicomp import multipletests
import h5py
import matplotlib.pyplot as plt
from scipy.spatial.distance import squareform
from collections import defaultdict
import sys
sys.path.append(os.path.abspath('../Python'))
import rsa_io
import rsa

In [2]:
# Path to write figure files to
figures_path = 'AgentPerception_Figures/New-22-03-2022/'

In [3]:
# Read mult regression results
path = "/auto/data2/oelmas/RSA_Results"
fname= "correlation_hamming_regression_results.pkl"
mult_reg_results_n_v, _, _,rsquared_n_v  = pd.read_pickle(path + '/Naive/Video/' + fname)
mult_reg_results_n_ff, _, _,rsquared_n_ff = pd.read_pickle(path + "/Naive/Still_FF/" + fname)
mult_reg_results_p_v, _, _,rsquared_p_s   = pd.read_pickle(path + "/Prior/Video/" + fname)
mult_reg_results_p_s, _, _,rsquared_p_v   = pd.read_pickle(path +  "/Prior/Still/" + fname)


# Read RSA results
fname= "correlation_hamming_avg_subjects_results.pkl"
rsa_results_n_v, _, _  = pd.read_pickle(path + "/Naive/Video/" +fname)
rsa_results_n_ff, _, _  = pd.read_pickle(path + '/Naive/Still_FF/' +fname)
rsa_results_p_v, _, _  = pd.read_pickle(path + '/Prior/Video/' +fname)
rsa_results_p_s, _, _  = pd.read_pickle(path + '/Prior/Still/' + fname)

# Create pandas results dataframes for multiple regression and rsa results 
mult_reg_results = [mult_reg_results_n_v, mult_reg_results_n_ff,mult_reg_results_p_v, mult_reg_results_p_s]
mult_reg_results = pd.concat(mult_reg_results)
rsa_results = [rsa_results_n_v, rsa_results_n_ff,rsa_results_p_v, rsa_results_p_s]
rsa_results = pd.concat(rsa_results)

In [4]:
# Dictionary for re-naming models in plots
model_name_dict = {"agent_hamming": "Agent Model",
                  "action_category_hamming":"Action Category Model"}

In [5]:
# Finds the peaks of kendall tau for each condition
def find_paek_kendall_tau(model="agent_hamming",er="parietal"):
    
    # Get model and electrode region rsa results
    mdl_idx = rsa_results.model_name==model
    er_idx = rsa_results.electrode_region==er
    mx_df = rsa_results[er_idx&mdl_idx]
    
    # get max kendall tau for each condition
    idx = mx_df.groupby(["experiment_type","stimulus_type"])["kendall_tau"].transform(max)==mx_df["kendall_tau"]
    return mx_df[idx]

In [6]:
# Given a dataframe with p values as a column, perform multiple comparison correction
def find_significance(df,p_value_column_name,method="bonferroni",alpha=0.05):
    
    # Finds significant time points, and adds the significance boolean values to the dataframe for rsa
    idx = multipletests(df["kendall_p-value"], alpha=alpha, method='bonferroni')[0]
    df.loc[:, 'significant'] = idx.tolist()
    return df.loc[df['significant']]

In [7]:
# Finds condinuous blocks of significance given the minimum number of points for a block
def find_continuous_sig_blocks(sig_points_list,min_npoint=5,difference_btw_points=2):
    cont_blocks=[]
    ind=0
    while ind < len(sig_points_list):
        size=0
        while ind+size+1 < len(sig_points_list) and sig_points_list[ind+size+1]-sig_points_list[ind+size] == difference_btw_points:
            size=size+1
        if(size+1>=min_npoint):
            cont_blocks.extend(sig_points_list[ind:ind+size+1])
        ind=ind+size+1
        
    return cont_blocks

In [7]:
# Compares Naive ff, mf and video rsa results in a figure, NOT USED RN
def compare_mf(kendall_tau_df,mult_reg_df,model, er, version):
    fig = go.Figure()
    colors = px.colors.qualitative.Plotly 
    
    # Still-ff still-mf video Naive comparison plot
    # Plot the lines from kendall tau df
    for i, cond in enumerate(["still-ff","still-mf","video"]):
    # Plots the kendall tau curve
        name = model
        if cond == "still-ff":
            if "gabor" in model:
                model = "ff_gabor_model_correlation"
                name="gabor"
            elif "intensity" in model:
                model = "ff_pixel_intensity_model_correlation"
                name = "pixel_intensity"
        elif cond == "still-mf":
            if "gabor" in model:
                model = "mf_gabor_model_correlation"
                name = "gabor"
            elif "intensity" in model:
                model = "mf_pixel_intensity_model_correlation"
                name = "intensity"
        elif cond == "video":
            if "gabor" in model:
                model = "video_gabor__frames_correlation"
                name = "gabor"
            elif "intensity" in model:
                model= "avg_pixel_intensity_model_correlation"
                name = "intensity"
        kendall_tau_df_to_plot = kendall_tau_df[(kendall_tau_df["model_name"] ==model) 
                                & (kendall_tau_df["electrode_region"] == er)
                                & (kendall_tau_df["experiment_type"] == "naive")]

        mult_reg_df_to_plot = mult_reg_df[(mult_reg_df["model_name"] == model) 
                                & (mult_reg_df["electrode_region"] == er)
                                    & (mult_reg_df["experiment_type"] == "naive")]
        # Not sure about the bonferroni, should we do bonferroni for each line or all lines (400 or 1200)
        sig_idx = multipletests(mult_reg_df_to_plot["p_value"]/2, alpha=0.001, method='bonferroni')[0]
        mult_reg_df_to_plot.loc[:, 'significant'] = sig_idx.tolist()
        sig_reg_df = mult_reg_df_to_plot.loc[mult_reg_df_to_plot['significant']]
        sig_reg_df.loc[:, 'plot_y'] = [-1]*len(sig_reg_df)
        sig_times = sig_reg_df[sig_reg_df.stimuli_type== cond]["time"].tolist()
        df_to_plot = kendall_tau_df_to_plot[kendall_tau_df_to_plot.stimulus_type== cond]
        fig.add_trace(go.Scatter(x=df_to_plot["time"], y=df_to_plot["kendall_tau"],
                                legendgroup=cond,
                                mode='lines',
                                name=cond,
                                line_color = colors[i]))
        fig.add_trace(go.Scatter(mode='markers',legendgroup=cond,
                                x=sig_times,
                                y=[-.1-(i*.02) for _ in range(len(sig_times))],
                                marker=dict( size=2,color= colors[i]),
                                showlegend=False))
    plotly.offline.plot(fig, filename = "/auto/data2/oelmas/RSA_Results_V"+str(version)+"/Naive/"+name+"_"+er+"_"+str(version)+"_still-mf_comparison.html", auto_open=False)
    fig.write_image("/auto/data2/oelmas/RSA_Results_V"+str(version)+"/Naive/"+name+"_"+er+"_"+str(version)+"_still-mf_comparison.png")

In [8]:
# ABOUT MIDDLE FRAME DONT RUN

# Plots for all models parietal and occipital channels naive: still-ff, still-mf, video For version 2#
#arg_dict = {model : ["parietal","occipital"] for model in v2_rsa_results["model_name"].unique() if "motion" not in model}
#for model, electrode_region_list in arg_dict.items():
#    for electrode_region in electrode_region_list:    
#        compare_mf(v2_rsa_results,v2_mult_reg_results,model,electrode_region,2)

NameError: name 'v2_rsa_results' is not defined

In [9]:
def inner(df_sorted):
    start_times=[]
    end_times=[]
    sig_wind_times=[]
    for i in range(df_sorted.shape[0]):
        if(df_sorted.iloc[i:-1].shape[0] > 5):
            if(( df_sorted.iloc[i+1] == df_sorted.iloc[i].time+2) and 
               ( df_sorted.iloc[i+2] == df_sorted.iloc[i+1].time+2) and
               ( df_sorted.iloc[i+3] == df_sorted.iloc[i+2].time+2) and
               ( df_sorted.iloc[i+4]== df_sorted.iloc[i+3].time+2) and
               ( df_sorted.iloc[i-1] != df_sorted.iloc[i].time-2)):
                start_times.append(df_sorted.iloc[i].time) 
        if(df_sorted.iloc[:i, :].shape[0] > 5):
            if(( df_sorted.iloc[i-1].time == df_sorted.iloc[i].time-2) and 
               ( df_sorted.iloc[i-2].time == df_sorted.iloc[i-1].time-2) and
               ( df_sorted.iloc[i-3].time == df_sorted.iloc[i-2].time-2) and
               ( df_sorted.iloc[i-4].time == df_sorted.iloc[i-3].time-2) and
               (df_sorted.shape[0] <=i+1 or df_sorted.iloc[i+1].time != df_sorted.iloc[i].time+2 )):
                end_times.append(df_sorted.iloc[i].time) 
        
    return start_times, end_times

SyntaxError: invalid syntax (<ipython-input-9-75af2fd5197f>, line 9)

In [8]:
# Condition comparison (Naive still, naive moving, prior still prior moving) 
# given a model and electrode region
def compare_conditions(reg_results, rsa_df, mdl="agent_hamming",er= "parietal"):
    inch_width = 5
    inch_height = 3
    dpi = 300

    width = inch_width * dpi
    height = inch_height * dpi
    
    
    condition_map = {
        "naive": "Naive",
        "still": "Still",
        "prior": "Prior",
        "video": "Moving"
    }
    
    
    # Using reg get the interested regression results for this plot
    mdl_idx = mult_reg_results.model_name==mdl
    er_idx = mult_reg_results.electrode_region==er
    vid_type_idx = mult_reg_results.stimuli_type=="video"
    still_type_idx = mult_reg_results.stimuli_type=="still-ff"
    reg_df = mult_reg_results[er_idx & mdl_idx & (still_type_idx | vid_type_idx)]

     # Using rsa_results get the action or agent model for all conditions and whole brain
    mdl_idx = rsa_df.model_name==mdl
    er_idx = rsa_df.electrode_region==er
    vid_type_idx = rsa_df.stimulus_type=="video"
    still_type_idx = rsa_df.stimulus_type=="still-ff"
    rsa_df = rsa_df[er_idx & mdl_idx & (still_type_idx | vid_type_idx)]
    
    # Significance testing for regression results
    sig_idx = multipletests(reg_df["p_value"], alpha=0.05, method='bonferroni')[0]
    reg_df.loc[:, 'significant'] = sig_idx.tolist()
    sig_reg_df = reg_df.loc[reg_df['significant']]

    #sig_reg_df = find_significance(reg_df, "p_value")
    # Compares conditions of one model
    fig = go.Figure()
    line_type = {("naive","still"):{"color": "orange",'dash': 'dash','width':4},
                 ("naive","video"):{"color": "orange",'dash': 'solid','width':4},
                 ("prior","still"):{"color":"blue",'dash': 'dash','width':4},
                 ("prior","video"):{"color":"blue",'dash': 'solid','width':4}}
    y_fix = 0
    for exp_type in rsa_df.experiment_type.unique():
        for stim_type in rsa_df.stimulus_type.unique():
            if not (stim_type =="still-mf"):

                action_df = rsa_df[(rsa_df.experiment_type == exp_type)
                                         &(rsa_df.stimulus_type == stim_type)]

                sig_time_reg_list = sig_reg_df[(sig_reg_df.experiment_type == exp_type)&
                                      (sig_reg_df.stimuli_type == stim_type)].time.to_list()
                
                sig_time_reg_filtered = find_continuous_sig_blocks(sig_time_reg_list)
                
                if stim_type=="video" and exp_type == "prior":
                    sig_time_ret = sig_time_reg_filtered
                    sig_time_old = sig_time_reg_list
                
                if(stim_type == "still-ff"):
                    stim_type = "still"
                
                fig.add_trace(go.Scatter(
                    x=action_df.time,
                    y=action_df.kendall_tau,
                    name=f'{condition_map[exp_type]}\u2014{condition_map[stim_type]}',
                    line=line_type[(exp_type,stim_type)],
                    showlegend=True))
        
               
                sig_time_drw = []
                for time in action_df.time:
                    if time not in sig_time_reg_filtered:
                        sig_time_drw.append(None)
                    else:
                        sig_time_drw.append(time)

                # Instead of empty dots vs filled dots maybe
                fig.add_trace(go.Scatter(
                                #name=exp_type +  " " + stim_type,
                                name=f'{condition_map[exp_type]}\u2014{condition_map[stim_type]}',
                                x=sig_time_drw,
                                y=[-.1-(y_fix*.02) if _ is not None else None for _ in sig_time_drw],
                                line=line_type[(exp_type,stim_type)]
                                ,showlegend=False))
                y_fix += 1
    # Adds the horizontal line  at t=0
    fig.add_shape(
        type='line',
        x0=0,
        y0=-0.2,
        x1=0,
        y1=0.6,
        line=dict(
            color='Black',
            width=8,
            dash= 'dash'
            
        ))
    # Change time range
    fig.update_yaxes(range=[-0.2, 0.6])
    
    # Change figure height and width
    fig.update_layout(height=height, width=width,title=er + " " +mdl,
         title_text="Correlations Between " + model_name_dict[mdl] + " and Conditions  on " + er +" Channels",
        xaxis_title="Time (ms, relative to stimulus onset)",
        yaxis_title="Kendall-tau Correlation",
        font=dict(size=50,color="#000000"),
                     legend=dict(itemwidth=50))
    
    # Write figure
    filename = figures_path+'condition_comparison/'+mdl+'_'+er
    fig.write_image(filename+'normal.svg', width=width, height=height, scale=1)
    #fig.write_image(filename+'.eps', width=width, height=height, scale=2)
    # plotly.offline.plot(fig, filename = filename+'.html', auto_open=False)
    return sig_time_old, sig_time_ret

In [9]:
compare_conditions(mult_reg_results, rsa_results, mdl="agent_hamming",er= "parietal")

/auto/k2/oelmas/anaconda/envs/agent_perception_rsa/lib/python3.6/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/auto/k2/oelmas/anaconda/envs/agent_perception_rsa/lib/python3.6/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



([60,
  70,
  72,
  90,
  92,
  94,
  96,
  98,
  100,
  102,
  104,
  106,
  108,
  110,
  112,
  114,
  116,
  118,
  120,
  122,
  124,
  126,
  128,
  130,
  132,
  134,
  136,
  138,
  140,
  142,
  144,
  146,
  148,
  150,
  152,
  154,
  156,
  158,
  160,
  162,
  164,
  166,
  168,
  170,
  172,
  222,
  224,
  226,
  228,
  230,
  232,
  234,
  236,
  238,
  240,
  242,
  260,
  262,
  264,
  266,
  268],
 [90,
  92,
  94,
  96,
  98,
  100,
  102,
  104,
  106,
  108,
  110,
  112,
  114,
  116,
  118,
  120,
  122,
  124,
  126,
  128,
  130,
  132,
  134,
  136,
  138,
  140,
  142,
  144,
  146,
  148,
  150,
  152,
  154,
  156,
  158,
  160,
  162,
  164,
  166,
  168,
  170,
  172,
  222,
  224,
  226,
  228,
  230,
  232,
  234,
  236,
  238,
  240,
  242,
  260,
  262,
  264,
  266,
  268])

In [ ]:
compare_conditions(mult_reg_results, rsa_results, mdl="agent_hamming",er= "parietal")
compare_conditions(mult_reg_results, rsa_results, mdl="agent_hamming",er= "occipital")
compare_conditions(mult_reg_results, rsa_results, mdl="agent_hamming",er= "whole_brain")
compare_conditions(mult_reg_results, rsa_results, mdl="agent_hamming",er= "frontal")
compare_conditions(mult_reg_results, rsa_results, mdl="action_category_hamming",er= "frontal")

In [ ]:
# Builds EEG data in (default=50) X ms time windows and calculates RDMs
# Specific time window rdm and a time windows overall rdm
def windowed_rdm(time_window=(50,100),experiment_type="naive", stimuli_type="still-ff",er="parietal",
                 eeg_root_path="/auto/data2/oelmas/EEG_AgentPerception_Naive/Still_FF/"):
    
    time_window_size = time_window[1] - time_window[0] +1
    
    # Read the EEG RDMs to conocatenate and average to get a bigger window
    subject_folders = [name for name in os.listdir(eeg_root_path) if
                       os.path.isdir(eeg_root_path + name) and name.startswith("subj")]
    
    subject_folders.sort()
    n_subjects = len(subject_folders)

    electrode_region = er
    time_window_rep_concat = defaultdict(list)
    time_window_rep_concat_spesific_tm_sig = defaultdict(list)
   
    # For all subjects build eeg data and 
    for i, subject_folder in enumerate(subject_folders):
        subj_name = subject_folder[0:9]
        subj_path = eeg_root_path + subject_folder + "/" + electrode_region + "/action-mats/"

        time_window_representations = rsa_io.build_eeg_data(subj_path, time_window_size, subj_name,
                                                            experiment_type, stimuli_type)

        time_window_representations_stw_sig = rsa_io.build_eeg_data_stw(subj_path, time_window, subj_name,
                                                            experiment_type, stimuli_type)

        for window, eeg_data in time_window_representations.items():    
            time_window_rep_concat[window].append(rsa.create_rdm(eeg_data,"correlation", "test"))

        time_window_rep_concat_spesific_tm_sig[time_window].append(
            rsa.create_rdm(time_window_representations_stw_sig[time_window],"correlation", "test"))

    # Convert windowed_eeg_rdm_dict from defaultdict to dict
    time_window_rep_concat = dict(time_window_rep_concat)
    
    for window, eeg_rdm_list in time_window_rep_concat.items():
        time_window_rep_concat[window] = np.vstack(eeg_rdm_list)

    time_window_rep_concat_spesific_tm_sig = np.vstack(time_window_rep_concat_spesific_tm_sig[time_window])
    return time_window_rep_concat, time_window_rep_concat_spesific_tm_sig

In [ ]:
time_window_rep_concat,_ = windowed_rdm()

# Displays the 50 ms time windowed RSA f,gures
n_subplots = len(time_window_rep_concat.keys())
fig, axs = plt.subplots(1,len(time_window_rep_concat.keys()),figsize=(4960/n_subplots,7016/n_subplots))
i=0
# traverse each window in time_window_representations and calculate rdm
for window, rdm_list in time_window_rep_concat.items():
    im = axs[i].imshow(squareform(np.mean(rdm_list,0)), interpolation='nearest')
    axs[i].set_title("Naive Still EEG-RDM between: "+ str((window[0],window[1])) + " ms",fontsize=30)
    i+=1
plt.savefig("rdm",dpi=600,format="pdf")

In [ ]:
# Plot 2x2 comparison 
# Tidy it up
def plot_experiment_stimuli_comparisons(rdm_df, electrode_region, model_name, mult_reg_results=None,version=2):
    fig = make_subplots(rows=2, cols=2,shared_yaxes=False,
    subplot_titles=("Naive: Still vs Video", "Prior: Still vs Video", 
                    "Still: Naive vs Prior", "Video: Naive vs Prior"))
    
    name=model_name
    
    # Show significance based on regression if given
    show_reg_significance = True
    if mult_reg_results is None:
        show_reg_significance = False
    colors = px.colors.qualitative.Plotly 

    variables = ['still-ff', 'video']
    constants = ['naive', 'prior']
    conditions = ["naive still-ff", "naive video", "prior still-ff", "prior video"]
    c_no = 0
    
    # Plots for Naive still vs Naive video, Prior still vs Prior video 
    for col_no, constant in enumerate(constants):
        for variable in variables:
            
            
            if variable == "still-ff":
                if "gabor" in model_name:
                    model_name = "ff_gabor_model_correlation"
                    name = "gabor"
                elif "intensity" in model_name:
                    model_name = "ff_pixel_intensity_model_correlation"
                    name = "intensity"
            elif variable == "video":
                if "gabor" in model_name:
                    model_name = "video_gabor_model_correlation"
                    name = "gabor"
                elif "intensity" in model_name:
                    model_name = "avg_pixel_intensity_model_correlation"
                    name = "intensity"
                    
                
            
            # Get rsa results by indexing with correct variables
            df_to_plot = rdm_df[(rdm_df["model_name"] == model_name) 
                                & (rdm_df["electrode_region"] == electrode_region)
                                & (rdm_df["experiment_type"] == constant) 
                                & (rdm_df["stimulus_type"] == variable)]
            
            if df_to_plot.empty:
                continue
                """
            # Plots the kendall tau curve
            fig.add_trace(go.Scatter(x=df_to_plot["time"],
                                    y=df_to_plot["kendall_tau"],
                                    legendgroup=constant + " " + variable,
                                    mode='lines',
                                    name=constant + " " + variable,
                                    line_color = colors[conditions.index(constant + " " + variable)]),
                                    row=1, col=col_no+1)
                                    """
           
            # Check significance based on regression results
            if(show_reg_significance):
               # legend_name = " regression significance"
                
                # Index the regression results based on constant(experiment type) and variable(stm type)
                mdl_idx = mult_reg_results.model_name==model_name
                ex_idx = mult_reg_results.experiment_type==constant
                st_idx = mult_reg_results.stimuli_type==variable
                er_idx = mult_reg_results.electrode_region==electrode_region
                reg_df = mult_reg_results[ex_idx & st_idx & er_idx & mdl_idx]
                
                if reg_df.empty:
                    print("Empty df: " +constant+"_"+variable+"_"+electrode_region + "_" + model_name)
                else:
                    # Multiple comparisons correction 
                    # TODO: Check if one tailed or two tailed, Divided p values by two to convert to one tailed
                    sig_idx = multipletests(reg_df["p_value"]/2, alpha=0.001, method='bonferroni')[0]
                    reg_df.loc[:, 'significant'] = sig_idx.tolist()
                    sig_reg_df = reg_df.loc[reg_df['significant']]
                    print( constant+"_"+variable+"_"+electrode_region + "_" + model_name,"-", len(sig_reg_df)*2)
                    
            # Check regression based on rsa p values
            else:
                
                #legend_name = " kendall tau significance"
                # Multiple comparisons correction 
                # Using two tailed 
                
                sig_idx = multipletests(df_to_plot["p_value"], alpha=0.001, method='bonferroni')[0]
                reg_df.loc[:, 'significant'] = sig_idx.tolist()
                sig_reg_df = reg_df.loc[reg_df['significant']]
                
            # Plotting the significance points in below the curves
            """fig.add_trace(go.Scatter(
                            mode='markers',
                            legendgroup=constant + " " + variable,
                            name=constant + " " + variable + legend_name,
                            x=sig_reg_df["time"].tolist(),
                            y=[-.1-(c_no*.02) for _ in range(len(sig_reg_df["time"].tolist()))],
                            marker=dict( size=2,color= colors[conditions.index(constant + " " + variable)]),
                            showlegend=False),row = 1, col=col_no+1)
            
            
            fig.update_yaxes(range=[-0.2, 0.6])
            fig.update_yaxes(title_text="Kendall-tau", row=1, col=col_no+1)
            fig.update_xaxes(title_text="Time ms (relative to stimulus onset)",  row=1, col=col_no+1)
            c_no = c_no+1
"""
     # Plots for Prior Still vs Naive Still, Naive video vs Prior video 
    constants = ['still-ff', 'video']
    variables = ['naive','prior']
    c_no = 0
    for col_no, constant in enumerate(constants):
        for  variable in variables:
            
            if constant == "still-ff":
                if "gabor" in model_name:
                    model_name = "ff_gabor_model_correlation"
                    name = "gabor"
                elif "intensity" in model_name:
                    model_name = "ff_pixel_intensity_model_correlation"
                    name="intensity"
            elif constant == "video":
                if "gabor" in model_name:
                    name= "gabor"
                    model_name = "video_gabor_model_correlation"
                elif "intensity" in model_name:
                    model_name = "avg_pixel_intensity_model_correlation"
                    name="intensity"
                    
            df_to_plot = rdm_df[(rdm_df["model_name"] == model_name) 
                                & (rdm_df["electrode_region"] == electrode_region)
                                & (rdm_df["experiment_type"] == variable) 
                                & (rdm_df["stimulus_type"] == constant)]
            if df_to_plot.empty:
                continue
            # Plotting the kendall tau curve
            """fig.add_trace(go.Scatter(x=df_to_plot["time"],
                                    y=df_to_plot["kendall_tau"],
                                    mode='lines',
                                    legendgroup=variable + " " + constant,
                                    name= variable + " " + constant,
                                    line_color = colors[conditions.index(variable + " " + constant)]),
                                    row=2, col=col_no+1)
"""
            
            # Check significance based on regression results
            if(show_reg_significance):
                #legend_name = " regression significance"
                
                # Index the regression results based on constant(experiment type) and variable(stm type)
                mdl_idx = mult_reg_results.model_name==model_name
                ex_idx = mult_reg_results.experiment_type==variable
                st_idx = mult_reg_results.stimuli_type==constant
                er_idx = mult_reg_results.electrode_region==electrode_region
                reg_df = mult_reg_results[ex_idx & st_idx & er_idx & mdl_idx]
                if reg_df.empty:
                    print("Empty df: " +constant+"_"+variable+"_"+electrode_region + "_" + model_name)
                else:
                    # Multiple comparisons correction 
                    # TODO: Check if one tailed or two tailed, Divided p values by two to convert to one tailed
                    sig_idx = multipletests(reg_df["p_value"]/2, alpha=0.001, method='bonferroni')[0]
                    reg_df.loc[:, 'significant'] = sig_idx.tolist()
                    sig_reg_df = reg_df.loc[reg_df['significant']]
                    print( constant+"_"+variable+"_"+electrode_region + "_" + model_name,"-", len(sig_reg_df)*2)
                    
            # Check regression based on rsa p values
            else:
                
                legend_name = " kendall tau significance"
                # Multiple comparisons correction 
                # TODO: Check if one tailed or two tailed, Divided p values by two to convert to one tailed
                
                sig_idx = multipletests(df_to_plot["p_value"]/2, alpha=0.001, method='bonferroni')[0]
                reg_df.loc[:, 'significant'] = sig_idx.tolist()
                sig_reg_df = reg_df.loc[reg_df['significant']]
            
            # Significance dots plotting
            fig.add_trace(go.Scatter(mode='markers',
                            legendgroup=variable + " " + constant,
                            name=variable + " " + constant,
                            x=sig_reg_df["time"].tolist(),
                            y=[-.1-(c_no*.02) for _ in range(len(sig_reg_df["time"].tolist()))],
                            marker=dict(size=2,color= colors[conditions.index(variable + " " + constant)]),
                            showlegend=False),row = 2, col=col_no+1)
            fig.update_yaxes(range=[-0.2, 0.6])
            fig.update_yaxes(title_text="Kendall-tau", row=1, col=col_no+1)
            c_no = c_no+1
    fig.update_layout(height=3508, width=2480,title=electrode_region + " " + model_name,
    xaxis_title="Time ms (relative to stimulus onset)",
    yaxis_title="Kendall-tau",
    font=dict(size=18,color="#000000"))
    plotly.offline.plot(fig, filename = "/auto/data2/oelmas/RSA_Results_V"+str(version)+"/"+electrode_region + "_" + name +'_experiment_stimuli_comparison.html', auto_open=False)
    fig.write_image("/auto/data2/oelmas/RSA_Results/"+electrode_region + "_" + name +'_experiment_stimuli_comparison.png')


In [ ]:
# Plots for all models parietal and occipital channels Naive, Prior x Still-ff, Video for version 2
arg_dict = {model : ["parietal","occipital","whole_brain",'frontal'] for model in rsa_results["model_name"].unique()   }
for model, electrode_region_list in arg_dict.items():
    for electrode_region in electrode_region_list:    
        plot_experiment_stimuli_comparisons(rsa_results,electrode_region, model, mult_reg_results,2)


In [ ]:
# Plots for all models parietal and occipital channels Naive, Prior x Still-ff, Video for version 1
arg_dict = {model : ["parietal","occipital","whole_brain",'frontal'] for model in v1_rsa_results["model_name"].unique()   }
for model, electrode_region_list in arg_dict.items():
    for electrode_region in electrode_region_list:    
        plot_experiment_stimuli_comparisons(v1_rsa_results,electrode_region, model, v1_mult_reg_results,1)



In [ ]:
# rename to electrode site comparison comparison plots
def plot_channels(model_name,rsa_results_df,regression_df):
    colors = px.colors.qualitative.Plotly
    fig = make_subplots(rows=2, cols=2,shared_yaxes=True,
                        subplot_titles=("Naive Still ", "Naive Video", 
                                        "Prior Still", "Prior Video"))
    er_list = rsa_results_df["electrode_region"].unique().tolist()
    mdl_idx = regression_df.model_name==model_name
    for row_no, exp_type in enumerate(['naive', 'prior']):
        for col_no, stm_type in enumerate(['still-ff', 'video']):
            
            # Index the RSA results with model, exp, stm
            df = rsa_results_df[(rsa_results_df["model_name"] == model_name) &
                                 (rsa_results_df["experiment_type"] == exp_type) &
                             (rsa_results_df["stimulus_type"] == stm_type)]
            
            # index the regression results with exp stm and mdl
            ex_idx = regression_df.experiment_type==exp_type
            st_idx = regression_df.stimuli_type==stm_type
            #reg_df = regression_df[ex_idx & st_idx & mdl_idx]
            for i, er in enumerate(er_list):
                if (er=="whole_brain"):
                    er_name="Whole Brain"
                    color = "black"
                else:
                    er_name = er.capitalize()
                    color = colors[er_list.index(er)]
                    
                if row_no == 0 and col_no == 0:
                    df_er = df[df["electrode_region"]==er]
                    fig.add_trace(go.Scatter(mode='lines',
                            x=df_er["time"],
                            y=df_er["kendall_tau"], 
                            line_color=color,line=dict(width=5),
                        name=er_name),
                        row = row_no+1, col=col_no+1)
                else:
                    df_er = df[df["electrode_region"]==er]
                    fig.add_trace(
                        go.Scatter(
                            mode='lines',
                            x=df_er["time"],
                            y=df_er["kendall_tau"], 
                            line_color=color,line=dict(width=5),
                            legendgroup=er,
                            showlegend=False,
                        name=er_name),
                        row = row_no+1, col=col_no+1)         
               
                # Also all use channels for bonferroni
                # To use only time for bonferroni correciton
                #er_idx = regression_df.electrode_region==er
                reg_df = regression_df[ex_idx & st_idx & mdl_idx]
                
                # Right now using two tailed
                sig_idx = multipletests(reg_df.p_value, alpha=0.05, method='bonferroni')[0]
                reg_df.loc[:, 'significant'] = sig_idx.tolist()
                sig_reg_df = reg_df.loc[reg_df['significant']]
                sig_time = sig_reg_df[sig_reg_df.electrode_region == er].time.tolist()
                sig_time_filtered = find_continuous_sig_blocks(sig_time)
                fig.add_trace(
                    go.Scatter(
                        mode='markers',
                        name=er_name,
                        x=sig_time_filtered,
                        y=[-.1-(i*.02) for _ in range(len(sig_time_filtered))],
                        marker=dict(
                            size=5,
                            color= color),
                        showlegend=False),
                    row = row_no+1, col=col_no+1)
    fig.update_yaxes(range=[-0.2, 0.6])
    fig.update_yaxes(title_text="Kendall-tau Correlation", row=row_no+1, col=col_no+1)
    fig.update_xaxes(title_text="Time (ms, relative to stimulus onset)",  row=row_no+1, col=col_no+1)
    fig.update_annotations(font_size=30)
    fig.update_layout(height=3508, width=2480, 
                      title="Correlations Between " + model_name_dict[model_name] + "RDM and EEG RDMS from Different Channels",
                     font=dict(size=30,color="#000000"),
                     legend=dict(itemwidth=30))
    plotly.offline.plot(fig, filename = figures_path + 'channel_comparison/'+model_name+'_channel_wise_comparison.html', auto_open=False)
    fig.write_image( figures_path + 'channel_comparison/'+model_name+'_channel_wise_comparison.pdf')

In [ ]:
plot_channels("action_category_hamming",rsa_results,mult_reg_results)
plot_channels("agent_hamming",rsa_results,mult_reg_results)

In [ ]:

stimuli_list =["robot-drink","robot-grasp","robot-handwave","robot-talk","robot-nudge",
        "robot-paper","robot-turn","robot-wipe","android-drink","android-grasp","android-handwave",
        "android-talk","android-nudge","android-paper","android-turn","android-wipe","human-drink",
        "human-grasp","human-handwave","human-talk","human-nudge","human-paper","human-turn","human-wipe"]

In [ ]:
# Given a Electrode Site, compare all models on different conditions
g_time={}
er="parietal"
colors = px.colors.qualitative.Plotly
fig = make_subplots(rows=2, cols=2,shared_yaxes=True,
                    subplot_titles=("Naive Still ", "Naive Video","Prior Still", "Prior Video"))
models = list(rsa_results["model_name"].unique())
for row_no, exp_type in enumerate(['naive', 'prior']):
    for col_no, stm_type in enumerate(['still', 'video']):
        if(exp_type=="naive" and stm_type=="still"):
            stm_type="still-ff"
        df = rsa_results[(rsa_results["electrode_region"] == er)&
                    (rsa_results["experiment_type"] == exp_type)&
                    (rsa_results["stimulus_type"] == stm_type)]
        df = find_significance(df,"kendall_p-value")
        for model in models:
            sig_time = {}
            df_model = df[df["model_name"]==model]
            sifig = go.Figure()
            
            fig.add_trace(
                go.Scatter(mode='lines',
                    x=df_model["time"],
                    y=df_model["kendall_tau"], 
                    line_color=colors[models.index(model)],
                           line=dict(size=8)
                    legendgroup=model,name=model),
                row = row_no+1, col=col_no+1
            ) 
            fig.add_trace(
                go.Scatter(
                    mode='markers',
                    legendgroup=model,
                    name=model,
                    x=sig_time[model],
                    y=[-.1-(i*.02) for _ in range(len(sig_time[model]))],
                    marker=dict(size=2,color= colors[models.index(model)]),showlegend=False),row = row_no+1, col=col_no+1)
            fig.update_yaxes(range=[-0.5, 0.8])
            fig.update_yaxes(title_text="Kendall-tau", row=row_no+1, col=col_no+1)
            fig.update_xaxes(title_text="Time ms (relative to stimulus onset)",  row=row_no+1, col=col_no+1)
    fig.update_layout(height=3508, width=2480,title = er,font=dict(size=50,color="#000000"),
                     legend=dict(itemwidth=50))
    plotly.offline.plot(fig, filename = "RSA_Results/test_plots/"+er+'_model_comparison.html', auto_open=False)

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

# Plots Model RDMs as heatmap
# TODO: use this function in other places to draw heatmap RDMs 
def plot_rdm_heatmap(RDM, title, fname, stimuli_list,vmin=0,vmax=1):
    fig = plt.figure(figsize=( 8-1/4 ,11-3/4),dpi=300)
    ax = fig.add_subplot()
    img=ax.imshow(squareform(RDM), interpolation='nearest',vmin=vmin, vmax=vmax)
    ax.set_xticks(np.arange(len(stimuli_list)))
    ax.set_yticks(np.arange(len(stimuli_list)))
    
    # ... and label them with the respective list entries
    ax.set_xticklabels(stimuli_list,fontsize=10)
    ax.set_yticklabels(stimuli_list,fontsize=10)
    for tick in ax.get_xticklabels():
        tick.set_rotation(90)
    
    plt.title(title, fontsize=30)
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.05)

    plt.colorbar(img, cax=cax)
    plt.tight_layout()
    plt.savefig(figures_path+"RDMS/"+fname+".pdf")

In [ ]:
model_path="/auto/data2/oelmas/RSA_Results/Naive/Video/modelRDMs/"
still_model_path="/auto/data2/oelmas/RSA_Results/Naive/Still_FF/modelRDMs/"
agent_model = np.load(model_path+"agent_hamming.npy")
action_model = np.load(model_path+"action_category_hamming.npy") 
pixel_intensity_avg = np.load(model_path+"avg_pixel_intensity_model_correlation.npy") 
motion = np.load(model_path+"pixel_wise_motion_model_correlation.npy") 
pixel_intensity = np.load(still_model_path+"ff_pixel_intensity_model_correlation.npy") 
  

In [ ]:
plot_rdm_heatmap(agent_model,"Agent Model RDM", "agent_model_rdm",stimuli_list)

plot_rdm_heatmap(action_model,"Action Model RDM", "action_model_rdm",stimuli_list)
plot_rdm_heatmap(pixel_intensity_avg,"Average Pixel Intensity RDM", "pixel_intensity_avg_rdm",stimuli_list)
plot_rdm_heatmap(motion,"Average Pixel Motion RDM", "avg_motion_rdm",stimuli_list)
plot_rdm_heatmap(pixel_intensity,"Pixel Intensity RDM", "pixel_intensity",stimuli_list)





In [ ]:
electrode_region="parietal"
windowed_eeg_rdm_dict_n_sff = {}
with h5py.File("/auto/data2/oelmas/RSA_Results/Naive/Still_FF/eegRDMs"+
               "/eeg_rdm_naive_still-ff_"+electrode_region+"_2_correlation.hdf5", "r") as f:
    attributes = list(f.attrs.items())
    for key in list(f.keys()):
          windowed_eeg_rdm_dict_n_sff[tuple(map(int, key[1:-1].split(',')))] = np.asarray(f[key]["rdm"])
            
windowed_eeg_rdm_dict_p_sff = {}
with h5py.File("/auto/data2/oelmas/RSA_Results/Prior/Still/eegRDMs"+
               "/eeg_rdm_prior_still-ff_"+electrode_region+"_2_correlation.hdf5", "r") as f:
    attributes = list(f.attrs.items())
    for key in list(f.keys()):
          windowed_eeg_rdm_dict_p_sff[tuple(map(int, key[1:-1].split(',')))] = np.asarray(f[key]["rdm"])

windowed_eeg_rdm_dict_n_v = {}      
with h5py.File("/auto/data2/oelmas/RSA_Results/Naive/Video/eegRDMs"+
               "/eeg_rdm_naive_video_"+electrode_region+"_2_correlation.hdf5", "r") as f:
    attributes = list(f.attrs.items())
    for key in list(f.keys()):
          windowed_eeg_rdm_dict_n_v[tuple(map(int, key[1:-1].split(',')))] = np.asarray(f[key]["rdm"])

windowed_eeg_rdm_dict_p_v = {}
with h5py.File("/auto/data2/oelmas/RSA_Results/Prior/Video/eegRDMs"+
               "/eeg_rdm_prior_video_"+electrode_region+"_2_correlation.hdf5", "r") as f:
    attributes = list(f.attrs.items())
    for key in list(f.keys()):
          windowed_eeg_rdm_dict_p_v[tuple(map(int, key[1:-1].split(',')))] = np.asarray(f[key]["rdm"])

In [ ]:
# Given windowed EEG_RDM_dict with keys time windows and values as EEG RDMs corresponding to those widows
# Draws EEG rdms for a given time window if time window is not given, draws the EEG RDM for the time window with max Kendall tau
def eeg_rdm_plot_heatmap(rsa_results,windowed_eeg_rdm_dict,electrode_region, model_name, stimuli_list, exp_type, stim_type,time_window=None,min_or_max=None):
    
    # We can give the pixel size we want and prevent scaling
    df_rsa = rsa_results[(rsa_results["electrode_region"] == electrode_region)&
                (rsa_results["model_name"] == model_name)]
    if not time_window:
        if min_or_max=="min":
            after_baseline = df_rsa[df_rsa.time.gt(0)]
            time=after_baseline.iloc[df_rsa.kendall_tau.argmin()].time
            time_window = (time,time+2)
        elif min_or_max=="max":
            time=df_rsa.iloc[df_rsa.kendall_tau.argmax()].time
            time_window = (time,time+2)
    eeg_rdm = np.mean(windowed_eeg_rdm_dict[time_window],0)
    print(df_rsa[df_rsa.time==time_window[0]])
    title = "Grand Avrage EEG RDM of "+ exp_type + " " + stim_type +" at \n" + str(time_window[0]) + "-" + str(time_window[1]) + " ms in " + electrode_region +" channels"
    fname = exp_type + "_" + stim_type + str(time_window[0]) + "-" + str(time_window[1]) +electrode_region
    plot_rdm_heatmap(eeg_rdm,title,fname,stimuli_list)

In [ ]:
eeg_rdm_plot_heatmap(rsa_results_n_ff,windowed_eeg_rdm_dict_n_sff,"parietal","agent_hamming",stimuli_list,"Naive","Still",min_or_max="max")
eeg_rdm_plot_heatmap(rsa_results_n_ff,windowed_eeg_rdm_dict_n_sff,"parietal","agent_hamming",stimuli_list,"Naive","Still",(-100,-98))
eeg_rdm_plot_heatmap(rsa_results_n_ff,windowed_eeg_rdm_dict_n_sff,"parietal","agent_hamming",stimuli_list,"Naive","Still",min_or_max="min")


In [ ]:
# All conditions EEG RDMs at -100, -98
t1 = -100
t2 = -98
eeg_rdm_plot_heatmap(rsa_results_n_ff,windowed_eeg_rdm_dict_n_sff,"parietal","agent_hamming",stimuli_list,"Naive","Still",(t1,t2))
eeg_rdm_plot_heatmap(rsa_results_n_v,windowed_eeg_rdm_dict_n_v,"parietal","agent_hamming",stimuli_list,"Naive","Video",(t1,t2))
eeg_rdm_plot_heatmap(rsa_results_p_s,windowed_eeg_rdm_dict_p_sff,"parietal","agent_hamming",stimuli_list,"Prior","Still",(t1,t2))
eeg_rdm_plot_heatmap(rsa_results_p_v,windowed_eeg_rdm_dict_p_v,"parietal","agent_hamming",stimuli_list,"Prior","Video",(t1,t2))

In [ ]:
# All conditions EEG RDMs at -100, -98
t1 = 24
t2 = 26
eeg_rdm_plot_heatmap(rsa_results_n_ff,windowed_eeg_rdm_dict_n_sff,"parietal","agent_hamming",stimuli_list,"Naive","Still",(t1,t2))
eeg_rdm_plot_heatmap(rsa_results_n_v,windowed_eeg_rdm_dict_n_v,"parietal","agent_hamming",stimuli_list,"Naive","Video",(t1,t2))
eeg_rdm_plot_heatmap(rsa_results_p_s,windowed_eeg_rdm_dict_p_sff,"parietal","agent_hamming",stimuli_list,"Prior","Still",(t1,t2))
eeg_rdm_plot_heatmap(rsa_results_p_v,windowed_eeg_rdm_dict_p_v,"parietal","agent_hamming",stimuli_list,"Prior","Video",(t1,t2))

In [ ]:
t1 = 110
t2 = 112
eeg_rdm_plot_heatmap(rsa_results_n_ff,windowed_eeg_rdm_dict_n_sff,"parietal","agent_hamming",stimuli_list,"Naive","Still",(t1,t2))
eeg_rdm_plot_heatmap(rsa_results_n_v,windowed_eeg_rdm_dict_n_v,"parietal","agent_hamming",stimuli_list,"Naive","Video",(t1,t2))
eeg_rdm_plot_heatmap(rsa_results_p_s,windowed_eeg_rdm_dict_p_sff,"parietal","agent_hamming",stimuli_list,"Prior","Still",(t1,t2))
eeg_rdm_plot_heatmap(rsa_results_p_v,windowed_eeg_rdm_dict_p_v,"parietal","agent_hamming",stimuli_list,"Prior","Video",(t1,t2))

In [ ]:
t1 = 400
t2 = 402
eeg_rdm_plot_heatmap(rsa_results_n_ff,windowed_eeg_rdm_dict_n_sff,"parietal","agent_hamming",stimuli_list,"Naive","Still",(t1,t2))
eeg_rdm_plot_heatmap(rsa_results_n_v,windowed_eeg_rdm_dict_n_v,"parietal","agent_hamming",stimuli_list,"Naive","Video",(t1,t2))
eeg_rdm_plot_heatmap(rsa_results_p_s,windowed_eeg_rdm_dict_p_sff,"parietal","agent_hamming",stimuli_list,"Prior","Still",(t1,t2))
eeg_rdm_plot_heatmap(rsa_results_p_v,windowed_eeg_rdm_dict_p_v,"parietal","agent_hamming",stimuli_list,"Prior","Video",(t1,t2))

In [ ]:
for i in range(-200,600,100):
    eeg_rdm_plot_heatmap(rsa_results_n_ff_2,windowed_eeg_rdm_dict_n_sff,"parietal","agent_hamming",stimuli_list,(i,i+2))

In [ ]:
for i in range(-200,600,100):
    eeg_rdm_plot_heatmap(rsa_results_n_v_2,windowed_eeg_rdm_dict_n_v,"parietal","agent_hamming",stimuli_list,(i,i+2))

In [ ]:
for i in range(-200,600,100):
    eeg_rdm_plot_heatmap(rsa_results_p_v_2,windowed_eeg_rdm_dict_p_v,"parietal","agent_hamming",stimuli_list,(i,i+2))

In [ ]:
for i in range(-200,600,100):
    eeg_rdm_plot_heatmap(rsa_results_p_s_2,windowed_eeg_rdm_dict_p_s,"parietal","agent_hamming",stimuli_list,(i,i+2))

In [ ]:
# Plots kendall tau traces for each subject in the same plot. Creates 4 plots (for 4 conditions)
def plot_seperated_subjects(seperated_subjects_dataframe, model_name, electrode_region):
    colors = px.colors.qualitative.Dark24
    fig = make_subplots(rows=2, cols=2,shared_yaxes=False,
                        subplot_titles=("Naive Still ", "Naive Video", 
                                        "Prior Still", "Prior Video"))
    for row_no, exp_type in enumerate(['naive', 'prior']):
        for col_no, stm_type in enumerate(['still', 'video']):
            color_no=0
            df_to_plot = seperated_subjects_dataframe[(seperated_subjects_dataframe["experiment_type"] == exp_type) &
            (seperated_subjects_dataframe["stimulus_type"] == stm_type)&
            (seperated_subjects_dataframe["model_name"] == model_name)&
            (seperated_subjects_dataframe["electrode_region"] == electrode_region)]
            
            for subject in subjects:
                fig.add_trace(go.Scatter(x=df_to_plot[df_to_plot["subject_no"]==subject]["time"],
                        y=df_to_plot[df_to_plot["subject_no"]==subject]["kendall_tau"],
                        mode="lines",
                        name="Subject" +str(subject),
                        line_color=colors[color_no],
                        legendgroup=str(subject)),
                        row = row_no+1, col=col_no+1) 
                color_no = color_no + 1
                    
    fig.update_layout(
        height=1000, width=1000,
        title= model_name + electrode_region)                
    fig.show()

In [ ]:
plot_seperated_subjects(seperated_subjects_data,"agent_hamming","occipital")